# Web Mining Exercise_Best Practice Solution

Task 1: Go to http://quotes.toscrape.com/ and scrap the following information from the website. The website as several pages.

![exercise01.PNG](images/exercise01.PNG)

* The quote
* The Author
* The tags


Task 2: Store the results in a database, retrieve them and create histogram on the number of quotes per author.

In [ ]:
# Import the two libraries which have to be used for scraping
# The request library is used to request information from the internet
import requests


# BeautifulSoup is a library allowing a code to parse through a html text
from bs4 import BeautifulSoup as bsoup 

# Requesting the website that contains the quotes, to work with it 
website = requests.get("http://quotes.toscrape.com") 

# Structure the website content to see how it is build and how the code blocks are structured
soup = bsoup(website.content)

# Print the structured website content to analyze it 
soup.prettify


In [ ]:
# Define the URL from which we want to scrape the information
url = "http://quotes.toscrape.com/page/%i/" 

# As the website has 10 pages in total we start at page 1 
page = 1 

# Parameter to stop our crawl when necessary
stop = False 

# List to include scraped info into
list1 = [] 

# Requesting webpage
while stop == False:
    r = requests.get("http://quotes.toscrape.com/page/%i/" % page)
    soup = bsoup(r.content)
    
   # If the crawler does not find a quote it should stop
    if soup.find("div", {"class":"quote"}) == None:
        stop = True
        
    # If a code is found, the following code is executed    
    else:
        
        # A generator is crawling over the website using the information given in the html-code
        # The crawler is looking for specific classes and delimiter for the different categories, which can be seen in the html code: 
            # A quote has the class "text" and has "span" as a delimiter
            # The author has the class "author" and has "small" as a delimiter
            # The tags have the class "tag" and have "a" as a delimiter
            # As there can be multiple tags for one quote, a second generator is used to extract all tags
        for quote in soup.find_all("div", {"class":"quote"}):
            d = {} # dictionary for our scraped information, where the data is stored
            d['quote'] = quote.find("span", {"class":"text"}).text
            d['author'] = quote.find("small", \
            {"class":"author"}).text
            tag_list = [tag.text for tag in quote.find_all("a",\
            {"class":"tag"})]
            d['tags'] = ', '.join(tag_list)
            
            # The crawled data is added to the list as a dictionary
            list1.append(d)
        
        # When one page is crawled the crawler starts with the next page by increasing the page number by one, until no more quotes are found 
        page += 1

In [ ]:
# Check if data is correctly structured to be imported into a sql-database using pandas dataframe
import pandas as pd
pd.DataFrame(list1)

In [ ]:
# Change list of dictionaries into a list of tuples to upload it to the database
result = [(d['quote'], d['author'], d['tags']) for d in list1]

#Show result
print(result)

In [ ]:
# Log-in information

# Import pymysql to connect to sql-database
import pymysql

# Define the information that is required to connect to database
host="xxx"
user="xxxx"
password="xxx"
db="xxx"



In [ ]:
# Table-creation in sql-database

# Connect to sql-database
connection = pymysql.connect(host=host,user=user,password=password,db=db)
try:
    cursorObject            = connection.cursor()  

# Add a table to the wm_grp2 database with an unique identifier (id) which increments automatically and create columns for the scraped quotes, authors and tags
    sqlCreateTableCommand  = "CREATE TABLE "+db+".myquotes ("\
        "id INT NOT NULL AUTO_INCREMENT,"\
        "quote VARCHAR(10000) NOT NULL,"\
        "author VARCHAR(100) NOT NULL,"\
        "tags VARCHAR(100) NOT NULL ,"\
        "PRIMARY KEY (id));"
    

# Execute the command to create the table in the sql-database
    cursorObject.execute(sqlCreateTableCommand)

    # Fetch all the rows - from the command output
    rows = cursorObject.fetchall()
    for row in rows:
        print(row)

# Define a message that will be shown if the code could not be executed correctly 
except Exception as e:
    print("Exeception occured:{}".format(e))

# Close connection to sql-database    
finally:
    connection.close()

In [ ]:
# Upload of scraped data to the created table in the sql-database

# Define an insert query where the values from the list can be passed into the table in the sql-database
insert_query = "INSERT INTO myquotes (quote, author, tags) VALUES (%s, %s, %s)"

# Connect to the sql-database
connection = pymysql.connect(host=host,user=user,password=password,db=db)
try:
    cursorObject = connection.cursor()                                     

    #Define the insert query as insertStatement
    insertStatement = insert_query
 
    # Using the executemany-statement to upload every element in the list of tuples to the sql-database
    cursorObject.executemany(insertStatement,result)
    connection.commit();

# Define a message that will be shown if the code could not be executed correctly   
except Exception as e:
    print("Exeception occured:{}".format(e))

# Close connection to sql-database
finally:
    connection.close()

In [ ]:
# Download of the information that is stored in the table in the sql-database

# Connect to the sql-database
connection = pymysql.connect(host=host,user=user,password=password,db=db)
try:
    cursorObject = connection.cursor() 
    
    # Select everything from the table 
    sqlQuery = "select * from "+db+".myquotes"
    cursor = connection.cursor()
    cursor.execute(sqlQuery)
    # Get all rows
    records = cursor.fetchall()
        
    # Print all data
    print(records)
    
    # Print data (structured in different rows) for better overview and to check if all information is included 
    print("\nPrinting table")
    for row in records:
        print("ID = ", row[0], )
        print("Quote = ", row[1], )
        print("Author = ", row[2])
        #"\n" to have space between results
        print("Tag  = ", row[3], "\n")
        
# Close connection to sql-database
finally:
    connection.close()

In [ ]:
# Store the downloaded data

# Store the data from the sql-database as a table (dataframe)
records_table = pd.DataFrame(records)

# Rename the columns
records_table.columns = ['ID', 'Quote', 'Author', 'Tags']

# Hide the index to improve data-appearance and show the data as a table (dataframe)
records_table.style.hide_index()

In [ ]:
records_table['Author'].value_counts().plot(kind='bar',figsize=(10,10))